In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import VGG16
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


In [3]:
# Load and preprocess CIFAR-10 data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step


In [4]:
# One-hot encode the labels
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)


In [5]:
# Preprocess images (resize and normalize)
def preprocess_images(images, labels):
    images = tf.image.resize(images, (224, 224)) / 255.0  # Resize to 224x224 and normalize
    return images, labels

batch_size = 32
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds = train_ds.map(preprocess_images).batch(batch_size).prefetch(tf.data.AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_ds = test_ds.map(preprocess_images).batch(batch_size).prefetch(tf.data.AUTOTUNE)


In [6]:
# Load the VGG16 model without the top layer
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
vgg_base.trainable = False  # Freeze the base layers initially

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [7]:
# Add a custom classification head
model = models.Sequential([
    vgg_base,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')  # CIFAR-10 has 10 classes
])


In [8]:
# Compile the model with initial hyperparameters
learning_rate = 0.001
model.compile(optimizer=optimizers.Adam(learning_rate=learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [9]:
# Define callbacks for early stopping and learning rate reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)


In [10]:
# Train the model
history = model.fit(train_ds,
                    epochs=20,
                    validation_data=test_ds,
                    callbacks=[early_stopping, reduce_lr])


Epoch 1/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 244s 147ms/step - accuracy: 0.2246 - loss: 2.0825 - val_accuracy: 0.4629 - val_loss: 1.4687 - learning_rate: 0.0010
Epoch 2/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 251s 149ms/step - accuracy: 0.3118 - loss: 1.7666 - val_accuracy: 0.5390 - val_loss: 1.3585 - learning_rate: 0.0010
Epoch 3/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 258s 146ms/step - accuracy: 0.3347 - loss: 1.7017 - val_accuracy: 0.5786 - val_loss: 1.3148 - learning_rate: 0.0010
Epoch 4/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 265s 148ms/step - accuracy: 0.3357 - loss: 1.6786 - val_accuracy: 0.5921 - val_loss: 1.2889 - learning_rate: 0.0010
Epoch 5/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 262s 148ms/step - accuracy: 0.3435 - loss: 1.6670 - val_accuracy: 0.5495 - val_loss: 1.3188 - learning_rate: 0.0010
Epoch 6/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 233s 149ms/step - accuracy: 0.3542 - loss: 1.6402 - val_accuracy: 0.5629 - val_loss: 1.2936 - learning_rate: 0.0010
Epoch 7/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 258s 146ms

In [11]:
# Unfreeze some layers for fine-tuning
vgg_base.trainable = True  # Unfreeze the entire base model


In [12]:
# Recompile with a smaller learning rate for fine-tuning
model.compile(optimizer=optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [13]:
# Fine-tune the model
history_fine_tune = model.fit(train_ds,
                              epochs=10,
                              validation_data=test_ds,
                              callbacks=[early_stopping, reduce_lr])



Epoch 1/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 838s 510ms/step - accuracy: 0.1084 - loss: 2.3640 - val_accuracy: 0.1000 - val_loss: 2.3568 - learning_rate: 1.0000e-04
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 786s 485ms/step - accuracy: 0.0999 - loss: 2.3520 - val_accuracy: 0.1000 - val_loss: 2.3379 - learning_rate: 1.0000e-04
Epoch 3/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 759s 486ms/step - accuracy: 0.0999 - loss: 2.3347 - val_accuracy: 0.1000 - val_loss: 2.3250 - learning_rate: 1.0000e-04
Epoch 4/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 806s 488ms/step - accuracy: 0.0999 - loss: 2.3229 - val_accuracy: 0.1000 - val_loss: 2.3164 - learning_rate: 1.0000e-04
Epoch 5/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 801s 488ms/step - accuracy: 0.0999 - loss: 2.3151 - val_accuracy: 0.1000 - val_loss: 2.3109 - learning_rate: 1.0000e-04


In [14]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_ds)
print(f"Final Test Accuracy: {test_acc:.2f}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 38s 122ms/step - accuracy: 0.0995 - loss: 2.3567
Final Test Accuracy: 0.10
